In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
Pkg.add(["CairoMakie", "LinearAlgebra", "Statistics", "Distributions", "Convex", "SCS"])

In [ ]:
using CairoMakie, LinearAlgebra, Statistics, Distributions, Convex, SCS

In [ ]:
x = LinRange(0,4,100)

In [ ]:
col_list = Vector{Float64}[]
for p in 0:19
    push!(col_list, x .^ p)
end
A = reduce(hcat,col_list)

In [ ]:
# alternative way of constructing the matrix
A2 = zeros(100,20)
for i in 1:100
    for j in 1:20
        A2[i,j] = x[i]^(j-1)
    end
end

In [ ]:
y = x .^ 2

In [ ]:
Ainv = pinv(A)

In [ ]:
y1 = y .+ rand(Normal(0,0.1),length(y))
b1 = Ainv * y1
barplot(b1) 

In [ ]:
fig = Figure()
ax = Axis(fig[1,1])
lines!(ax,x,y)
plot!(ax,x,y1)
fig

In [ ]:
function fitlq(A, y)
    Ainv = pinv(A)
    return Ainv * y
end

function lasso(A, y, λ1, λ2)
    x = Variable(size(A)[2])
    problem = minimize(norm(A * x - y, 2) + λ1 * norm(x, 1) + λ2 * norm(x, 2))
    solve!(problem, SCS.Optimizer; silent = true)
    return vec(evaluate(x))
end

In [ ]:
b10ls = lasso(A[:, 1:10], y1, 0.1, 0)
b10lq = fitlq(A[:, 1:10], y1)

In [ ]:
barplot(b10ls)
barplot!(b10lq)
current_figure()

In [ ]:
# Perform 100 realizations for both least squares and Lasso on 20th-degree polynomial

coeff_lsq_all = []
coeff_lasso_all = []

for i in 1:100
    y_noisy = y .+ rand(Normal(0, 0.1), length(y))
    coeff_lsq = fitlq(A, y_noisy)
    coeff_lasso = lasso(A, y_noisy, 0.1, 0.01)
    push!(coeff_lsq_all, coeff_lsq)
    push!(coeff_lasso_all, coeff_lasso)
end

In [ ]:
fig_14a = Figure()
ax_14a = Axis(fig_14a[1, 1], title="Variance of Least Squares Coefficients",
              xlabel="Coefficient Index", ylabel="Variation")
boxplot!(ax_14a, hcat(coeff_lsq_all)..., color=:blue)
fig_14a

In [ ]:
fig_14c = Figure()
ax_14c = Axis(fig_14c[1, 1], title="Variance of Lasso Coefficients",
              xlabel="Coefficient Index", ylabel="Variation")
boxplot!(ax_14c, hcat(coeff_lasso_all)..., color=:purple)
fig_14c

In [ ]:
errors_lsq = []
errors_lasso = []

for i in 1:100
    y_noisy = y .+ rand(Normal(0, 0.1), length(y))
    pred_lsq = A * fitlq(A, y_noisy)
    pred_lasso = A * lasso(A, y_noisy, 0.1, 0.01)
    push!(errors_lsq, mean((y_noisy - pred_lsq) .^ 2))
    push!(errors_lasso, mean((y_noisy - pred_lasso) .^ 2))
end

In [ ]:
fig_15a = Figure()
ax_15a = Axis(fig_15a[1, 1], title="Error Comparison for Least Squares and Lasso",
              xlabel="Regression Method", ylabel="Error")
boxplot!(ax_15a, [errors_lsq, errors_lasso], color=:green)
ax_15a.xticks = (1:2, ["Least Squares", "Lasso"])
fig_15a


In [ ]:
fig_15b = Figure()
ax_15b = Axis(fig_15b[1, 1], title="Least Squares Error Distribution",
              xlabel="Realization", ylabel="Error")
boxplot!(ax_15b, errors_lsq, color=:blue)
fig_15b

In [ ]:
fig_15c = Figure()
ax_15c = Axis(fig_15c[1, 1], title="Lasso Error Distribution",
              xlabel="Realization", ylabel="Error")
boxplot!(ax_15c, errors_lasso, color=:purple)
fig_15c